In [1]:
%pylab inline 
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
import sys
sys.path.insert(0,'/Users/dl264294/Desktop/github/flowpm/')
#from flowpm.spectrum_for_cosmology import power_spectrum_for_cosmology
from flowpm.tfbackground import cosmo,z2a,a2z,afactor, chifactor
from flowpm.tfpower import linear_matter_power
from flowpm.angular_power_tf import measure_power_spectrum_tf
from flowpm.raytracing import  lightcone, Born
import flowpm 


Populating the interactive namespace from numpy and matplotlib


In [2]:
from flowpm.tfpower import linear_matter_power
from scipy.interpolate import InterpolatedUnivariateSpline as iuspline
klin = np.loadtxt('/Users/dl264294/Desktop/github/flowpm/flowpm/data/Planck15_a1p00.txt').T[0]
plin=linear_matter_power(cosmo, klin)
ipklin = iuspline(klin, plin)

In [3]:

nc=[64,64,640]   # size of the cube, number of cells
plane_size=64                    # number of pixel for x and  y 
Boxsize=[200,200,2000]          # Physical size of the cube
r = np.linspace(0,2000,10, endpoint=True)
a = afactor(r)  
a_s=z2a(1.00)
ds=chifactor(a_s)
field=5.
a0=0.1
af=1.0
n_steps=10
@tf.function
def power_spectrum_for_cosmology(
              Omega0_m,
              sigma8):
    cosmology=cosmo
    cosmology['Omega0_m']=tf.convert_to_tensor(Omega0_m,dtype=tf.float32)
    cosmology['sigma8']=tf.convert_to_tensor(sigma8,dtype=tf.float32)
    init_stages = np.linspace(a0, af, n_steps, endpoint=True)
    initial_conditions = flowpm.linear_field(nc,    
                                            Boxsize, 
                                             ipklin,         
                                             batch_size=1)
    # Sample particles
    state = flowpm.lpt_init(initial_conditions, 0.1)   
    # Evolve particles down to z=0
    final_state = flowpm.nbody(state, init_stages, nc)         
    # Retrieve final density field
    state, lps_a, lps=lightcone(final_state, a[::-1], 
                                  nc, 
                                    field*60/plane_size, plane_size,
              cosmology)
    k_map=Born(lps_a,lps,ds,nc,Boxsize,plane_size,field,cosmology)
    k_map=tf.cast(k_map,dtype=tf.complex64)
    ell, power_spectrum=measure_power_spectrum_tf(k_map,field,plane_size)
    return ell, power_spectrum, k_map



In [4]:
ell3, power_spectrum3, kmap=power_spectrum_for_cosmology(0.3075,0.8159)

In [ ]:
Omega0_m= tf.constant(0.3075)
sigma8=tf.constant(0.8159)
with tf.GradientTape() as tape:
    tape.watch(Omega0_m)
    ell, power_spectrum,kmap= power_spectrum_for_cosmology(
              Omega0_m,
              sigma8)
dy_dx = tape.jacobian(power_spectrum, Omega0_m)
print(dy_dx)